In [ ]:
import os.path
import datetime

import numpy as np

import pygetm
import pygetm.input.gotm

gotm_cases = "../../../gotm-cases"

In [ ]:
lon, lat, depth = 1.283333, 59.33333, 110.0
x = np.linspace(-1000, 1000, 3)
y = np.linspace(-1000, 1000, 3)
domain = pygetm.domain.create_cartesian(x, y, nz=110, interfaces=True, H=depth, periodic_x=True, periodic_y=True, lat=lat, z0=0.03)

In [ ]:
radiation = pygetm.radiation.TwoBand(jerlov_type=pygetm.radiation.JERLOV_I)
airsea = pygetm.airsea.FluxesFromMeteo(humidity_measure=pygetm.HumidityMeasure.RELATIVE_HUMIDITY)
sim = pygetm.Simulation(domain, pygetm.BAROCLINIC, airsea=airsea, radiation=radiation, gotm=os.path.join(gotm_cases, "nns_annual/gotmturb.nml"))

In [ ]:
meteo = pygetm.input.gotm.get_meteo(os.path.join(gotm_cases, "nns_annual/meteo.dat"))
sim.airsea.u10.set(meteo.u10)
sim.airsea.v10.set(meteo.v10)
sim.airsea.t2m.set(meteo.t2m)
sim.airsea.rh.set(meteo.hum)
sim.airsea.sp.set(100 * meteo.sp)
sim.airsea.tcc.set(meteo.tcc)
sim.airsea.tp.set(0.0)

sim.temp.set(8.1)
sim.salt.set(35.14)

In [ ]:
nc = sim.output_manager.add_netcdf_file('gotm.nc', interval=datetime.timedelta(hours=1))
nc.request('temp', 'salt', 'sst', 'nuh')
sim.start(meteo.time.values[0], 360.0, 10, report=datetime.timedelta(days=10), report_totals=datetime.timedelta(days=1000))
while sim.time < meteo.time.values[-1]:
    sim.advance()
sim.finish()